# SASSIE Delta 18O Bottle Data
<p>Bottles for Delta 18O analysis were collected from the ice and waters periodically throughout the SASSIE cruise to determine influence of freshwater sources on the SASSIE study area. Bottle data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>

<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

<h1>Data Download and Metadata Viewing</h1>

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests

<h2>Download The Delta 18O Bottle data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Bottle data from PO.DAAC, which is accessed through earthdata. If you do not already have an earthdata account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your earthdata credentials below.</p>

In [ ]:
username = 'your_username'
password = 'your_password'
earthdata = requests.auth.HTTPBasicAuth(username, password)

##LOCAL DIRECTORY TO SAVE FLOAT DATA
dir_in = 'Data/delta18O/'

The next block of code creates the directory specified above and downloads the shipboard bottle data file to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
#If the directory specified above does not exist, it is created here.
if not os.path.exists(dir_in):
    os.makedirs(dir_in)
    
if not os.path.isfile(dir_in+'SASSIE_Fall_2022_Shipboard_Delta18O.nc'):
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_DELTA_18O_V1/SASSIE_Fall_2022_Shipboard_Delta18O.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(dir_in+'SASSIE_Fall_2022_Shipboard_Delta18O.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + dir_in+'SASSIE_Fall_2022_Shipboard_Delta18O.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard Delta 18O file is already in local directory')



<h2> View The Metadata Inside the Delta 18O Files</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 


In [ ]:
#See information about the entire dataset:
file = dir_in + 'SASSIE_Fall_2022_Shipboard_Delta18O.nc'
ds = xr.open_dataset(file)
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually ake figures from the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir = 'Figures/Delta18O/'

#Define the Delta symbol for use in figures: 
delta = "\u03B4"

#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var):
    if var =='d18O':
        colormap = 'rainbow'
        var_label = delta+'$^1$$^8$O'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin = 70, latmax =74,lonmin=-157,lonmax=-140):
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetry and/or AMSR ice data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for AMSR ice data and acess bathymetry data from NOAA
    <br>2. Download Ship Track data and define a function that will add it to the map fro reference. 
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for Ice Data and Access Bathymetry Data </h4>

In [ ]:
##LOCAL DIRECTORY TO SAVE AMSR SEA ICE DATA 
ice_dir = 'Ice_Data/'

#ICE DATA DIR
if not os.path.exists(ice_dir):
    os.makedirs(ice_dir)

##READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Add Ice Data to a Map</h4>
This function will download the AMSR ice product at the date specified specified and add it to the plot in blue. 

In [ ]:
def add_ice_data(ice_date):
        #get ice data at ice_date
        play_start = ice_date
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        i=0
        time_tmp=play_start - timedelta(days=i)
        while os.path.isfile(ice_dir+filename_si)==False and i<15:
            time_tmp=play_start - timedelta(days=i)
            filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(time_tmp.year)+str(time_tmp.month).zfill(2)+str(time_tmp.day).zfill(2)+'.he5'
            url = requests.get('https://n5eil01u.ecs.nsidc.org/AMSA/AU_SI12.001/'+str(time_tmp.year)+'.'+str(time_tmp.month).zfill(2)+'.'+str(time_tmp.day).zfill(2)+'/'+filename_si)
            with open(ice_dir+filename_si, 'wb') as f:
                f.write(url.content)
            i=i+1

        #apply ice data at ice_date in blue
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        if os.path.isfile(ice_dir+filename_si):
            dsc = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km')
            dsd = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km/Data Fields') 
            var1 = np.array(dsd.SI_12km_NH_ICECON_DAY.squeeze().values)
            indx = np.where((var1==120) | (var1==0))
            var1[indx] = 0
            ice_location = np.ma.masked_where(var1==0,var1)

            pp = ax.pcolormesh(dsc.lon,dsc.lat,ice_location, 
                            vmin=0,vmax=100, # Set max and min values for plotting
                            cmap='Blues_r', shading='auto',   # shading='auto' to avoid warning
                            transform=cartopy.crs.PlateCarree())
        

<h4>Create Directory for Shiptrack Data and Access Bathymetry Data </h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')
    
    
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin = 70, latmax =74,lonmin=-157,lonmax=-140):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code 

<h2>Make Maps of This Data</h2>
<p>Using the code in this section, the user can plot the bottle data on a map colored by time, sea surface temperature, or Delta 18O. The code blocks in the 'supporting code' section should be run first. The user will need to input a local directory to save figures and ice data for these to work properly. </p>

<h3> Mapping the Time and Location of Bottle Data Collection</h3>

The map_d18O_samples function shows the locations of bottle data collection on a map, colored by time. Various features of this function: 
    <br>1. This function adds the track of the R/V Woldstad for reference in black. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product at a specific date shown by inputting ice_data=datetime(yyyy,m,d) where yyyy,m,d is replaced with the date of interest. </p>

In [ ]:
##Map The D18O Samples and color by time. 
def map_d18o_samples(bathymetry_data=True,ice_date=False,shiptrack=True):
    
    ##LOAD DATA FOR SELECTED PLAY
    
    time_data = np.squeeze(ds['time'].values)
    lat_data = np.squeeze(ds['latitude'].values)
    lon_data = np.squeeze(ds['longitude'].values)
    time_ice_data = np.squeeze(ds['time_ice'].values)
    lat_ice_data = np.squeeze(ds['latitude_ice'].values)
    lon_ice_data = np.squeeze(ds['longitude_ice'].values)
    
    ##CREATE A MAP WITH LAND, CITY MARKERS

    map_study_area()
    
    #Give the Map a title 
    ax.set_title('Location and Time of '+delta+'$^1$$^8$O Sample Collection',fontsize=18) 
        

    ##OPTIONAL MAP ADD-ONs
    
    if bathymetry_data == True:    
        add_bathy_data()
        
    if ice_date !=False:
        ##ADD ICE DATA
        add_ice_data(ice_date)
        
    if shiptrack==True:
        add_ship_track()
        
        
    ##APPLY WATER LAT/LON DATA
    track = ax.scatter(lon_data, 
             lat_data,s = 30, 
             c=time_data,cmap='jet',
               transform=cartopy.crs.PlateCarree(),zorder=2)
    cbar = fig.colorbar(track, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(min(time_data).astype('int64'),max(time_data).astype('int64'),4))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)

    ##APPLY ICE LAT/LON DATA
    track = ax.scatter(lon_ice_data, 
             lat_ice_data,s = 30, 
             c=time_ice_data,cmap='jet',
               transform=cartopy.crs.PlateCarree(),marker = 'D',vmin = min(time_data).astype('int64'),vmax = max(time_data).astype('int64'),zorder=2)    
    
    #MAKE DUMMY ICE AND WATER MARKERS FOR LEGEND 
    ax.scatter(0,0,s=30,c='k',transform=cartopy.crs.PlateCarree(),label = 'Water Sample')
    ax.scatter(0,0,s=30,c='k',transform=cartopy.crs.PlateCarree(),marker='D',label = 'Ice Sample')
    plt.legend(loc=2)

    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'map'):
        os.makedirs(fig_dir+'map')
        print('Saving Output Image:  '+fig_dir+'map/SASSIE_DELTA_18O_LOCATION.png')
        plt.savefig(fig_dir+'map/SASSIE_DELTA_18O_LOCATION.png',dpi='figure',format='png')

In [ ]:
map_d18o_samples()
#map_d18o_samples(ice_date = datetime(2022,9,20))

<h3> Mapping the Temperature and Delta 18O of Bottle Data Collection</h3>

The map_d18O_samples function shows the locations of bottle data collection on a map, colored by a variable from the file. Various features of this function: 
    <br>1. This function adds the track of the R/V Woldstad for reference in black. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product at a specific date shown by inputting ice_data=datetime(yyyy,m,d) where yyyy,m,d is replaced with the date of interest. </p>

In [ ]:
##Map The Ship Track and contour by time 
def map_d18o_sample_var(var,bathymetry_data=True,ice_date = False,shiptrack=True):
    
    ##LOAD DATA
    time_data = np.squeeze(ds['time'].values)
    var_data = np.squeeze(ds[var].values)
    lat_data = np.squeeze(ds['latitude'].values)
    lon_data = np.squeeze(ds['longitude'].values)
    time_ice_data = np.squeeze(ds['time_ice'].values)
    var_ice_data = np.squeeze(ds[var+'_ice'].values)
    lat_ice_data = np.squeeze(ds['latitude_ice'].values)
    lon_ice_data = np.squeeze(ds['longitude_ice'].values)
    
    ##CREATE A MAP WITH LAND, CITY MARKERS, BATHYMETRY DATA, AND AMSR ICE DATA AT START AND END OF PLAY.
    
    #set a color map and variable label
    colormap,var_label = define_variable_attributes(var)
    
    #make the map 
    map_study_area()
    
    #add a title
    ax.set_title(var_label +' Measurements from Sample Collection',fontsize=18) 
        

    
    ##OPTIONAL MAP ADD-ONs
    
    if bathymetry_data == True:    
        add_bathy_data()
        
    if ice_date !=False:
        add_ice_data(ice_date)
        
    if shiptrack==True:
        add_ship_track()
        
    ##APPLY WATER LAT/LON DATA
    track = ax.scatter(lon_data, 
             lat_data,s = 30, 
             c=var_data,cmap=colormap,
               transform=cartopy.crs.PlateCarree(),zorder=2,vmin = min(var_data).astype('int64'),vmax = max(var_data).astype('int64'))
    matplotlib.pyplot.colorbar(track, ax=ax, orientation="horizontal", pad=0.1).set_label(label=var_label,size='large',weight='bold')
    if var=='d18O':
        ##APPLY ICE LAT/LON DATA
        track = ax.scatter(lon_ice_data, 
                 lat_ice_data,s = 30, 
                 c=var_ice_data,cmap=colormap,
                   transform=cartopy.crs.PlateCarree(),marker = 'D',vmin = min(var_data).astype('int64'),vmax = max(var_data).astype('int64'),zorder=2)    

    #MAKE DUMMY ICE AND WATER MARKERS FOR LEGEND 
    ax.scatter(0,0,s=30,c='k',transform=cartopy.crs.PlateCarree(),label = 'Water Sample')
    ax.scatter(0,0,s=30,c='k',transform=cartopy.crs.PlateCarree(),marker='D',label = 'Ice Sample')
    plt.legend(loc=4)

    ##SAVE FIGURE
    if not os.path.exists(fig_dir+var):
        os.makedirs(fig_dir+var)
    print('Saving Output Image:  '+fig_dir+var+'/SASSIE_DELTA_18O_LOCATION.png')
    plt.savefig(fig_dir+var+'/SASSIE_DELTA_18O_LOCATION.png',dpi='figure',format='png')

In [ ]:
#map_d18o_sample_var('d18O',ice_date = datetime(2022,9,15))
map_d18o_sample_var('temperature')

<h2> Making a Histogram of Delta 18O Ratio and Temperature</h2>

In [ ]:
##MAKE HISTOGRAM OF D18O Data 
def histogram_d18o(var):

    ##LOAD DATA
    d18O_data= np.squeeze(ds['d18O'].values)
    temp_data = np.squeeze(ds['temperature'].values)
     #MAKE A TWO PANNELED TIME SERIES PLOT
    fig,axs = plt.subplots(2)
    fig.tight_layout(pad=2.0)
    fig.suptitle('Histogram of ' +delta+'$^1$$^8$O Bottle Data')
    
 
 
    axs[0].hist(d18O_data,bins=20,facecolor='k',edgecolor='white',linewidth=3)
    axs[0].set(xlabel=delta+'$^1$$^8$O Ratio',ylabel='Number of \n Measurements')

    axs[1].hist(temp_data,bins=20,facecolor='k',edgecolor='white',linewidth=3)
    axs[1].set(xlabel='Temperature ($^{\circ}$C)',ylabel='Number of \n Measurements')
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'Histogram'):
        os.makedirs(fig_dir+'Histogram')
    print('Saving Output Image:  '+fig_dir+'Histogram'+'/SASSIE_DELTA_18O_Hist.png')
    plt.savefig(fig_dir+'Histogram'+'/SASSIE_DELTA_18O_Hist.png',dpi='figure',format='png')

In [ ]:
histogram_d18o('d18O')